In [88]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from datetime import date
from datetime import timedelta
from datetime import datetime

%run web_scraping.ipynb
%run useful_functions.ipynb

In [89]:
shows = get_shows()
show_broadcast_details = get_show_broadcast_details()
broadcast_details = get_broadcast_details()
broadcast_playlists = get_broadcast_playlists()

In [90]:
#shows_to_run = ["https://www.bbc.co.uk/programmes/m00008zy/episodes/guide", 
#                "https://www.bbc.co.uk/programmes/b03hjfww/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b006wkh9/episodes/guide",
#                "https://www.bbc.co.uk/programmes/p04m6srg/episodes/guide",
#                "https://www.bbc.co.uk/programmes/m000m5bw/episodes/guide",
#                "https://www.bbc.co.uk/programmes/m0016zh5/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b0072lb2/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b0072q60/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b00p4l4t/episodes/guide",
#                "https://www.bbc.co.uk/programmes/b0072l4x/episodes/guide"]

temp_show_broadcast_details = pd.DataFrame()

#for show_url in shows_to_run:
for show_url in shows['Show URL']:
    show_instance = Show(show_url)
    if shows.empty or show_instance.show_key not in shows['Show Key'].unique():
        shows = pd.concat([shows,show_instance.show_details])
    temp_show_broadcast_details = pd.concat([temp_show_broadcast_details,show_instance.broadcasts])
    
temp_show_broadcast_details['Datetime Key'] = datetime.now()
temp_show_broadcast_details['Is New Data'] = True
temp_show_broadcast_details['Is Maybe Pending'] = True

show_broadcast_details = pd.concat([show_broadcast_details,temp_show_broadcast_details])
show_broadcast_details = show_broadcast_details.groupby(allbut(show_broadcast_details.columns,['Datetime Key','Is New Data','Is Maybe Pending'])).apply(lambda group: group.nsmallest(1, columns='Datetime Key')).reset_index(drop=True)
show_broadcast_details = show_broadcast_details.groupby("Broadcast Key").apply(lambda group: group.nlargest(1, columns='Datetime Key')).reset_index(drop=True)

In [124]:
new_broadcast_data = show_broadcast_details[(show_broadcast_details["Is Broadcast Upcoming"]==False) & ((show_broadcast_details["Is New Data"]==True) | (show_broadcast_details['Is Maybe Pending'] == True))][0:10]
temp_broadcast_details = pd.DataFrame()
temp_broadcast_playlists = pd.DataFrame()

for i, [broadcast_url,broadcast_key] in enumerate(new_broadcast_data[['Broadcast URL','Broadcast Key']].values):
    broadcast_instance = Playlist(broadcast_url)
    if i % 10 == 0:
        print(f'Broadcast Number {1+i} of {len(new_broadcast_data.index)}')
    if (len(broadcast_instance.playlist.index) == 0) & ((pd.to_datetime('now')- broadcast_instance.broadcast_details["Service Date"]).dt.days.values <= 7):
        show_broadcast_details.loc[show_broadcast_details["Broadcast Key"] == broadcast_key,'Is Maybe Pending'] = True
    else:
        show_broadcast_details.loc[show_broadcast_details["Broadcast Key"] == broadcast_key,'Is Maybe Pending'] = False
    temp_broadcast_details = pd.concat([temp_broadcast_details,broadcast_instance.broadcast_details])
    temp_broadcast_playlists = pd.concat([temp_broadcast_playlists,broadcast_instance.playlist])
    
temp_broadcast_details['Datetime Key'] = datetime.now()
temp_broadcast_details['Is New Data'] = True
temp_broadcast_playlists['Datetime Key'] = datetime.now()
temp_broadcast_playlists['Is New Data'] = True

broadcast_details = pd.concat([broadcast_details,temp_broadcast_details])
broadcast_details = broadcast_details.groupby(allbut(broadcast_details.columns,['Datetime Key','Is New Data'])).apply(lambda group: group.nsmallest(1, columns='Datetime Key')).reset_index(drop=True)
broadcast_details = broadcast_details.groupby("Broadcast Key").apply(lambda group: group.nlargest(1, columns='Datetime Key')).reset_index(drop=True)

broadcast_playlists = pd.concat([broadcast_playlists,temp_broadcast_playlists])
broadcast_playlists = broadcast_playlists.groupby(allbut(broadcast_playlists.columns,['Datetime Key','Is New Data'])).apply(lambda group: group.nsmallest(1, columns='Datetime Key')).reset_index(drop=True)
broadcast_playlists = broadcast_playlists.groupby(["Broadcast Key", "Broadcast Position"]).apply(lambda group: group.nlargest(1, columns='Datetime Key')).reset_index(drop=True)


Broadcast Number 1 of 10


In [125]:
show_broadcast_details['Is New Data'] = False
broadcast_details['Is New Data'] = False
broadcast_playlists['Is New Data'] = False

shows.to_pickle("../data/shows.pkl")
show_broadcast_details.to_pickle("../data/show_broadcast_details.pkl")
broadcast_details.to_pickle("../data/broadcast_details.pkl")
broadcast_playlists.to_pickle("../data/broadcast_playlists.pkl")

In [126]:
mask_2022 = broadcast_details['Service Date'].apply(lambda date: date.year) == 2022

(
broadcast_details[mask_2022]
.merge(broadcast_playlists, on='Broadcast Key', how='left', suffixes=(" Broadcast Details",""))
.merge(show_broadcast_details, on='Broadcast Key', how='left', suffixes=(""," Show Details"))
.merge(shows, on='Show Key', how='left')
.sort_values(['Service Date','Broadcast Position'], ascending = [False, True])
)

,Show Title,Broadcast Banner,Broadcast Long Description,Service Image URL,Service Day,Service Date,Service Time,Broadcast Key,Datetime Key Broadcast Details,Is New Data Broadcast Details,...,Broadcast URL,Broadcast Title,Broadcast Short Description,Is Broadcast Upcoming,Show Key,Datetime Key Show Details,Is New Data Show Details,Is Maybe Pending,Show URL,Show Name
174,Radio 1's Power Down Playlist with Sian Eleri,Comfort Blanket,Soothing tracks to make you feel warm and cosy...,https://programmes.files.bbci.co.uk/programmes...,Mon 14 Nov 2022,2022-11-14,22:00,-9053511899857064598,2022-12-11 13:10:30.277778,False,...,https://www.bbc.co.uk/programmes/m001dvhb,Comfort Blanket,Soothing tracks to make you feel warm and cosy...,False,-3943587521332424384,2022-12-11 12:37:22.098745,False,False,https://www.bbc.co.uk/programmes/p04m6srg/epis...,Radio 1's Power Down Playlist with Sian Eleri
175,Radio 1's Power Down Playlist with Sian Eleri,Comfort Blanket,Soothing tracks to make you feel warm and cosy...,https://programmes.files.bbci.co.uk/programmes...,Mon 14 Nov 2022,2022-11-14,22:00,-9053511899857064598,2022-12-11 13:10:30.277778,False,...,https://www.bbc.co.uk/programmes/m001dvhb,Comfort Blanket,Soothing tracks to make you feel warm and cosy...,False,-3943587521332424384,2022-12-11 12:37:22.098745,False,False,https://www.bbc.co.uk/programmes/p04m6srg/epis...,Radio 1's Power Down Playlist with Sian Eleri
176,Radio 1's Power Down Playlist with Sian Eleri,Comfort Blanket,Soothing tracks to make you feel warm and cosy...,https://programmes.files.bbci.co.uk/programmes...,Mon 14 Nov 2022,2022-11-14,22:00,-9053511899857064598,2022-12-11 13:10:30.277778,False,...,https://www.bbc.co.uk/programmes/m001dvhb,Comfort Blanket,Soothing tracks to make you feel warm and cosy...,False,-3943587521332424384,2022-12-11 12:37:22.098745,False,False,https://www.bbc.co.uk/programmes/p04m6srg/epis...,Radio 1's Power Down Playlist with Sian Eleri
177,Radio 1's Power Down Playlist with Sian Eleri,Comfort Blanket,Soothing tracks to make you feel warm and cosy...,https://programmes.files.bbci.co.uk/programmes...,Mon 14 Nov 2022,2022-11-14,22:00,-9053511899857064598,2022-12-11 13:10:30.277778,False,...,https://www.bbc.co.uk/programmes/m001dvhb,Comfort Blanket,Soothing tracks to make you feel warm and cosy...,False,-3943587521332424384,2022-12-11 12:37:22.098745,False,False,https://www.bbc.co.uk/programmes/p04m6srg/epis...,Radio 1's Power Down Playlist with Sian Eleri
178,Radio 1's Power Down Playlist with Sian Eleri,Comfort Blanket,Soothing tracks to make you feel warm and cosy...,https://programmes.files.bbci.co.uk/programmes...,Mon 14 Nov 2022,2022-11-14,22:00,-9053511899857064598,2022-12-11 13:10:30.277778,False,...,https://www.bbc.co.uk/programmes/m001dvhb,Comfort Blanket,Soothing tracks to make you feel warm and cosy...,False,-3943587521332424384,2022-12-11 12:37:22.098745,False,False,https://www.bbc.co.uk/programmes/p04m6srg/epis...,Radio 1's Power Down Playlist with Sian Eleri
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,Steve Lamacq,Steve has live music from Bess Atwell,Steve kicks of the week with a Kitchen Disco t...,https://programmes.files.bbci.co.uk/programmes...,Mon 7 Feb 2022,2022-02-07,16:00,-9196240667197234118,2022-12-11 12:37:39.355677,False,...,https://www.bbc.co.uk/programmes/m001475c,Steve has live music from Bess Atwell,And the National Anthem search continues,False,1408665427283283890,2022-12-11 12:37:22.098745,False,False,https://www.bbc.co.uk/programmes/b0072lb2/epis...,Steve Lamacq
24,Steve Lamacq,Steve has live music from Bess Atwell,Steve kicks of the week with a Kitchen Disco t...,https://programmes.files.bbci.co.uk/programmes...,Mon 7 Feb 2022,2022-02-07,16:00,-9196240667197234118,2022-12-11 12:37:39.355677,False,...,https://www.bbc.co.uk/programmes/m001475c,Steve has live music from Bess Atwell,And the National Anthem search continues,False,1408665427283283890,2022-12-11 12:37:22.098745

In [127]:
shows
show_broadcast_details
broadcast_details
#broadcast_playlists

,Show Title,Broadcast Banner,Broadcast Long Description,Service Image URL,Service Day,Service Date,Service Time,Broadcast Key,Datetime Key,Is New Data
0,Guy Garvey's Finest Hour,Today's Theme - The Over Driven Organ,Elbow frontman Guy Garvey loves the Hammond or...,https://programmes.files.bbci.co.uk/programmes...,Sun 11 Nov 2018,2018-11-11,14:00,-9217409020754396024,2022-12-11 12:37:39.355677,False
1,Radio 1's Indie Show with Jack Saunders,"New Music from Tame Impala, Pulled Apart by Ho...",Jack plays you the best in indie and alternati...,https://programmes.files.bbci.co.uk/programmes...,Mon 28 Oct 2019,2019-10-28,23:00,-9210574728566079079,2022-12-11 12:37:39.355677,False
2,Steve Lamacq's Roundtable,Tom Ravenscroft sits in,Tom Ravenscroft hosts Thursday nights' roundta...,https://programmes.files.bbci.co.uk/programmes...,Thu 24 Aug 2017,2017-08-24,18:00,-9210072269533792936,2022-12-11 12:37:39.355677,False
3,Radio 1's Indie Show with Jack Saunders,Fickle Friends,Natti from Fickle Friends plays some of her al...,https://programmes.files.bbci.co.uk/programmes...,Thu 2 Dec 2021,2021-12-02,20:00,-9210012826913911293,2022-12-11 12:37:39.355677,False
4,Stuart Maconie's Freak Zone,Deathprod,"Stuart talks to Helge Sten, aka Deathprod, abo...",https://programmes.files.bbci.co.uk/programmes...,Sun 8 Dec 2019,2019-12-08,20:00,-9208310403030560793,2022-12-11 12:37:39.355677,False
5,Steve Lamacq,Steve has live music from Bess Atwell,Steve kicks of the week with a Kitchen Disco t...,https://programmes.files.bbci.co.uk/programmes...,Mon 7 Feb 2022,2022-02-07,16:00,-9196240667197234118,2022-12-11 12:37:39.355677,False
6,Steve Lamacq,Steve recaps the weekend's #6MusicFestival,Steve recaps the weekend's #6MusicFestival & h...,https://programmes.files.bbci.co.uk/programmes...,Mon 4 Apr 2022,2022-04-04,16:00,-9196141610708013740,2022-12-11 12:37:39.355677,False
7,Radio 1's Future Sounds with Clara Amfo,Holly Humberstone is the BRITs Rising Star,"The biggest and best new music in the world, i...",https://programmes.files.bbci.co.uk/programmes...,Thu 9 Dec 2021,2021-12-09,18:00,-9190739167977103050,2022-12-11 12:37:39.355677,False
8,Steve Lamacq's Roundtable,"Roundtable: Chris Hawkins, Ben Folds, Leonie C...",Steve hosts Roundtable with American singer-so...,https://programmes.files.bbci.co.uk/programmes...,Thu 15 Oct 2015,2015-10-15,18:00,-9188430697977792663,2022-12-11 12:37:39.355677,False
9,Guy Garvey's Finest Hour,Let's Have a Heated Debate,"Guy's back in his Brixton writing room, with m...",https://programmes.files.bbci.co.uk/programmes...,Sun 26 Jul 2020,2020-07-26,15:00,-9177229921058442386,2022-12-11 12:37:39.355677,False
